# WEB SCRAPING ASSIGNMENT 3

1. Write a python program which searches all the product under a particular product from
www.amazon.in. The product to be searched will be taken as input from user.

In [1]:
#import required libraries
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time
import re
import urllib.request

In [2]:
def search(term="Laptop"):
    search_xpath = '//*[@id="twotabsearchtextbox"]'
    search_button = wait.until(ec.element_to_be_clickable((By.XPATH, search_xpath)))
    search_button.send_keys(term)
    search_button.send_keys(Keys.ENTER)

if __name__=="__main__":
    term = input("Enter product : ")
    browser = webdriver.Chrome()
    browser.get("https://www.amazon.in")
    wait = WebDriverWait(browser, 600)
    search(term)

Enter product : Laptop


2.In the above question, now scrape the following details of each product listed in first 3 pages
of your search results and save it in a dataframe and csv. In case if any product has less than 3
pages in search results then scrape all the products available under that product vertical.
Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings",
"Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it
by “-“.

In [3]:
def search(term="Laptop"):
    search_xpath = '//*[@id="twotabsearchtextbox"]'
    search_button = wait.until(ec.element_to_be_clickable((By.XPATH, search_xpath)))
    search_button.send_keys(term)
    search_button.send_keys(Keys.ENTER)

def getProducts():
    products = soup.find_all("a", {"class":"a-link-normal s-no-outline"})
    if len(products)>0:
        return products
    return []

def clickNext(browser):
    xpath = '//li[@class="a-last"]'
    btn = wait.until(ec.element_to_be_clickable((By.XPATH, xpath)))
    btn.click()

def brandName(soup):
    try:
        return soup.find(text="Brand").findNext("td").get_text().strip()
    except:
        return "-"

def prodName(soup):
    try:
        data = soup.find("span",{"id":"productTitle"}).get_text()
        data = re.sub("\s+"," ",data)
        return data.strip()
    except:
        return "-"
def getRating(soup):
    try:
        data = soup.find("span",{"class":"a-icon-alt"}).get_text().split(" ")[0]
        if "Previous" in data:
            return "-"
        else:
            return data
    except:
        return "-"

def numRatings(soup):
    try:
        return soup.find("span",{"id":"acrCustomerReviewText"}).get_text().split(" ")[0]
    except:
        return "-"

def getPrice(soup):
    try:
        return soup.find("span",{"id":"priceblock_ourprice"}).get_text()
    except:
        return "-"

def returnExchange(soup):
    return bool(soup.find("a",{"class":"a-size-small a-link-normal a-text-normal"}).get_text())

def expectedDelivery(soup):
    try:
        div = soup.find("div",{"id":"ddmDeliveryMessage"})
        return div.find("b").get_text().strip()
    except:
        return "-"
def availability(soup):
    text = soup.get_text().lower()
    return bool(re.search("in stock", text))

def otherDetails(soup):
    try:
        data = soup.find("div",{"id":"productDescription"}).get_text()
        data = re.sub("\s+"," ",data)
        return data.strip()
    except:
        return "-"

def report(brand_name, prod_name, rating, num_of_ratings, price, return_exchange, expected_delivery, avail, other_details,prod_url):
    df = pd.read_excel("DataQ2.xlsx")

    #Saving data into excel sheet
    data = {"Brand Name":[brand_name], "Product Name":[prod_name],"Rating":[rating],
            "No. of Ratings":[num_of_ratings],"Price":[price],"Return/Exchange":[return_exchange],
            "Expected Delivery":[expected_delivery],"Availability":[avail],
            "Other Details":[other_details],"Product URL":[prod_url]
            }
    df = df.append(pd.DataFrame(data))
    writer = pd.ExcelWriter("DataQ2.xlsx")
    df.to_excel(writer,index=False)
    writer.save()

if __name__=="__main__":
    term = input("Enter product : ")
    browser = webdriver.Chrome()
    browser.get("https://www.amazon.in")
    wait = WebDriverWait(browser, 600)
    search(term)

    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")

    products = []
    for i in range(3):
        products=products+getProducts()
        try:
            clickNext(browser)
        except:
            continue

    for product in products:
        prod_url = "https://www.amazon.in"+product["href"]
        browser.get(prod_url)
        html = browser.page_source
        soup = BeautifulSoup(html,"html.parser")

        brand_name = brandName(soup)
        prod_name = prodName(soup)
        rating = getRating(soup)
        num_of_ratings = numRatings(soup)
        price = getPrice(soup)
        return_exchange = returnExchange(soup)
        expected_delivery = expectedDelivery(soup)
        avail = availability(soup)
        other_details = otherDetails(soup)
        print(brand_name)
        print(prod_name)
        print(rating)
        print(num_of_ratings)
        print(price)
        print(return_exchange)
        print(expected_delivery)
        print(avail)
        print(other_details)
        report(brand_name, prod_name, rating, num_of_ratings, price, return_exchange, expected_delivery, avail, other_details, prod_url)
    

Enter product : Laptop
Lenovo
Lenovo IdeaPad 3 (2021) | 11th Gen Intel Core i5 |15.6" Full HD Thin and Light Laptop | 4-Side Narrow Bezel (8GB/512GB SSD/Windows 10/MS Office 2019/Backlit Keyboard/Arctic Grey/1.65Kg), 82H8014BIN
3.6
4
-
True
Sunday, July 4
True
Pattern name:Lenovo IdeaPad 3 Engineered for long-lasting performance, the Lenovo IdeaPad 3 delivers powerful performance in a thin and light laptop—it’s perfect for your everyday tasks with features that you can depend on.
Lenovo
Lenovo IdeaPad 5 11th Gen Intel Core i5 15.6" FHD IPS Thin & Light Laptop (8GB/256GB SSD + 1TB HDD/Win 10/Office/Integrated Iris Xe Graphics/Fingerprint Reader/Graphite Grey/1.66Kg), 82FG010BIN
-
-
-
True
Sunday, July 4
True
Pattern name:Lenovo IdeaPad 5 The new Lenovo IdeaPad 5 boasts an enticing list of included and optional features, making it a powerful combination of performance, connectivity, and entertainment. Enjoy crisp visuals on the FHD display along with the rich sound of user-facing, Dolby 

3.Write a python program to access the search bar and search button on images.google.com and
scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [10]:
def search(query):
    search_xpath = '//input[@class="gLFyf gsfi"]'
    search_button = wait.until(ec.element_to_be_clickable((By.XPATH, search_xpath)))
    search_button.send_keys(query)
    search_button.send_keys(Keys.ENTER)

def getURLs(soup):
    div = soup.find("div",{"class":"islrc"})
    imgs = div.find_all("img",limit=120)
    urls = []
    for img in imgs:
        try:
            urls.append(img["src"])
        except:
            continue
    return urls

if __name__=="__main__":
    browser = webdriver.Chrome()
    wait = WebDriverWait(browser, 600)
    queries = ["fruits", "cars", "Machine Learning"]

    for query in queries:
        print("Now searching for {}".format(query))
        browser.get("https://images.google.com/")
        search(query)
        html = browser.page_source
        soup = BeautifulSoup(html, "html.parser")
        urls = getURLs(soup)
        for i,url in enumerate(urls):
            file_name = "C:/Users/Kavitha/Documents/10 Questions/images/"+query+str(i)+".jpg"
            urllib.request.urlretrieve(url, file_name)

Now searching for fruits
Now searching for cars
Now searching for Machine Learning


4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page.
Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”,
“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display
Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. In
case if any of the details is missing then replace it by “- “. Save your results in a dataframe
and CSV.

In [13]:
def brandName(soup):
    try:
        return soup.find("span",{"class":"B_NuCI"}).get_text().strip().split()[0]
    except:
        return "-"

def phoneName(soup):
    try:
        return soup.find("span",{"class":"B_NuCI"}).get_text().strip()
    except:
        return "-"

def color(soup):
    try:
        return soup.find(text="Color").findNext("td").get_text().strip()
    except:
        return "-"

def ram(soup):
    try:
        return soup.find(text="Ram Capacity").findNext("td").get_text().strip()
    except:
        try:
            return soup.find(text="RAM").findNext("td").get_text().strip()
        except:
            return "-"
def storage(soup):
    try:
        return soup.find(text="Storage Memory").findNext("td").get_text().strip()
    except:
        try:
            return soup.find(text="Internal Storage").findNext("td").get_text().strip()
        except:
            return "-"

def primaryCamera(soup):
    try:
        return soup.find(text="Primary Camera").findNext("td").get_text().strip()
    except:
        return "-"

def secondCamera(soup):
    try:
        return soup.find(text="Secondary Camera").findNext("td").get_text().strip()
    except:
        return "-"

def displaySize(soup):
    try:
        return soup.find(text="Display Size").findNext("td").get_text().strip()
    except:
        return "-"

def displayResol(soup):
    try:
        return soup.find(text="Resolution").findNext("td").get_text().strip()
    except:
        return "-"
def processor(soup):
    try:
        return soup.find(text="Processor Type").findNext("td").get_text().strip()
    except:
        return "-"

def cores(soup):
    try:
        return soup.find(text="Processor Core").findNext("td").get_text().strip()
    except:
        return "-"

def battery(soup):
    try:
        return soup.find(text="Battery Capacity").findNext("td").get_text().strip()
    except:
        return "-"

def price(soup):
    try:
        return soup.find("div",{"class":"_30jeq3 _16Jk6d"}).get_text().strip()
    except:
        return "-"
def report(brand, name, col, rm, rom, pri_cam, sec_cam, dis_size,
                des_res, proc, core, bat, ruppee, prod_url):
    df = pd.read_excel("C:/Users/Kavitha/Documents/10 Questions/Q4.xlsx")
    #Saving data into excel sheet
    data = {"Brand Name":[brand], "Product Name":[name],"Color":[col],
            "RAM":[rm],"ROM":[rom],"Primary Camera":[pri_cam],
            "Secondary Camera":[sec_cam],"Display Size":[dis_size],
            "Display Resolution":[des_res],"Processor":[proc],"Cores":[core],
            "Battery":[bat],"Price":[ruppee],"Product URL":[prod_url]
            }
    df = df.append(pd.DataFrame(data))
    writer = pd.ExcelWriter("Q4.xlsx")
    df.to_excel(writer,index=False)
    writer.save()
if __name__=="__main__":
    browser = webdriver.Chrome()
    browser.get("https://www.flipkart.com")
    wait = WebDriverWait(browser, 600)

    #close button
    xpath = '//button[@class="_2KpZ6l _2doB4z"]'
    btn = browser.find_element_by_xpath(xpath)
    try:
        btn.click()
    except:
        pass

    #search
    xpath = '//input[@title="Search for products, brands and more"]'
    btn = browser.find_element_by_xpath(xpath)
    btn.send_keys("Oneplus Nord")
    btn.send_keys(Keys.ENTER)

    #get all a's
    time.sleep(5)
    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")

    a_tags = soup.find_all("a",{"class":"_1fQZEK"})
    urls = []
    for a in a_tags:
        prod_url = "https://www.flipkart.com"+a["href"]
        urls.append(prod_url)

    #Iterate through each url
    for url in urls:
        browser.get(url)
        html = browser.page_source
        soup = BeautifulSoup(html, "html.parser")
        brand = brandName(soup)
        name = phoneName(soup)
        col = color(soup)
        rm = ram(soup)
        rom = storage(soup)
        pri_cam = primaryCamera(soup)
        sec_cam = secondCamera(soup)
        dis_size = displaySize(soup)
        des_res = displayResol(soup)
        proc= processor(soup)
        core = cores(soup)
        bat = battery(soup)
        ruppee = price(soup)

        report(brand, name, col, rm, rom, pri_cam, sec_cam, dis_size,
                des_res, proc, core, bat, ruppee, prod_url)
        print(brand, name, col, rm, rom, pri_cam, sec_cam, dis_size,
                des_res, proc, core, bat, ruppee, prod_url)
        print()
        print()

OnePlus OnePlus Y Series 80 cm (32 inch) HD Ready LED Smart Android TV  (32HA0A00) Black 1 GB 8 GB - - In The Box - - - - ₹15,499 https://www.flipkart.com/boult-audio-bassbuds-x1-wired-headset/p/itm6e33ec78f848e?pid=ACCFHWT5ZVBTYDDY&lid=LSTACCFHWT5ZVBTYDDYNPUEY3&marketplace=FLIPKART&q=Oneplus+Nord&store=search.flipkart.com&srno=s_1_40&otracker=search&otracker1=search&fm=SEARCH&iid=335666fd-5c95-4cc1-b2b3-d62b3cf792bc.ACCFHWT5ZVBTYDDY.SEARCH&ppt=hp&ppn=homepage&ssid=wnmp2wxxcw0000001624793998689&qH=53863d4d978d5074


OnePlus OnePlus U1S 126 cm (50 inch) Ultra HD (4K) LED Smart Android TV  (50UC1A00) - 2 GB 16 GB - - In The Box - - - - ₹39,999 https://www.flipkart.com/boult-audio-bassbuds-x1-wired-headset/p/itm6e33ec78f848e?pid=ACCFHWT5ZVBTYDDY&lid=LSTACCFHWT5ZVBTYDDYNPUEY3&marketplace=FLIPKART&q=Oneplus+Nord&store=search.flipkart.com&srno=s_1_40&otracker=search&otracker1=search&fm=SEARCH&iid=335666fd-5c95-4cc1-b2b3-d62b3cf792bc.ACCFHWT5ZVBTYDDY.SEARCH&ppt=hp&ppn=homepage&ssid=wnmp2wxxcw

5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on
google maps.

In [26]:
from selenium.webdriver import ActionChains

if __name__=="__main__":
    term = input("Enter city : ")
    browser = webdriver.Chrome()
    browser.get("https://www.google.com/maps")

    xpath = '//*[@id="searchboxinput"]'
    btn = browser.find_element_by_xpath(xpath)
    btn.send_keys(term)
    btn.send_keys(Keys.ENTER)
    
    time.sleep(5)

    body = browser.find_element_by_tag_name("body")
    actionChains = ActionChains(browser)
    actionChains.context_click(body).perform()

Enter city : delhi


6.Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –
September 20) from trak.in

In [21]:
if __name__=="__main__":
    browser = webdriver.Chrome()
    browser.get("https://trak.in/")
    wait = WebDriverWait(browser, 600)
    
    #Goto Funding Deals
    xpath = '//li[@id="menu-item-51510"]'
    btn = wait.until(ec.element_to_be_clickable((By.XPATH,xpath)))
    btn.click()

    time.sleep(5)
    html = browser.page_source
    soup = BeautifulSoup(html,"html.parser")

    #Get data for July
    table_july = soup.find("table",{"id":"tablepress-48"})
    df = pd.read_html(str(table_july))
    print("July")
    print(df)
    print()

    #Get data for August
    table_aug = soup.find("table",{"id":"tablepress-49"})
    df = pd.read_html(str(table_aug))
    print("August")
    print(df)
    print()

    #Get data for September
    table_sep = soup.find("table",{"id":"tablepress-50"})
    df = pd.read_html(str(table_sep))
    print("September")
    print(df)
    print()

July
[   Sr. No. Date (dd/mm/yyyy)               Startup Name  \
0        1        15/07/2020                   Flipkart   
1        2        16/07/2020                    Vedantu   
2        3        16/07/2020                       Crio   
3        4        14/07/2020                    goDutch   
4        5        13/07/2020                   Mystifly   
5        6        09/07/2020               JetSynthesys   
6        7        10/07/2020                   gigIndia   
7        8        15/07/2020                  PumPumPum   
8        9        14/07/2020                       FLYX   
9       10        13/07/2020  Open Appliances Pvt. Ltd.   

        Industry / Vertical  \
0                E-commerce   
1                   EduTech   
2                   EduTech   
3                   FinTech   
4       Airfare Marketplace   
5  Gaming and Entertainment   
6               Marketplace   
7         Automotive Rental   
8                OTT Player   
9    Information Technology   

  

7.Write  a program to scrap all the available details of best gaming laptops from digit.in

In [22]:
if __name__=="__main__":
    browser = webdriver.Chrome()
    wait = WebDriverWait(browser, 600)

    #Goto Homepage
    browser.get("https://www.digit.in/")

    #Goto Gaming Tab
    browser.get("https://www.digit.in/gaming/")

    #Goto Gaming mart tab
    browser.get("https://www.digit.in/gaming-mart/")

    #Goto Gaming Laptops
    browser.get("https://www.digit.in/gaming-mart/topten/gaming-laptops/")

    #Click on Read more button
    xpath = '//a[@class="review"]'
    btn = wait.until(ec.element_to_be_clickable((By.XPATH, xpath)))
    btn.click()

    time.sleep(3)

    html = browser.page_source
    soup = BeautifulSoup(html,"html.parser")

    #Get all product links
    urls = soup.find_all("a",{"class":"review spec"})

    for i,url in enumerate(urls,start=1):
        print("Getting data for product {}".format(i))
        print()
        browser.get(url["href"])
        time.sleep(3)
        html = browser.page_source
        soup = BeautifulSoup(html,"html.parser")

        details = soup.find("div",{"class":"tabcontent"}).get_text().strip()
        details = re.sub("\s\s+","\n",details)
        details = re.sub("\s:\s",":",details)

        print(details)
        print()
        print()

Getting data for product 1

Basic Information
Model Name:Nitro 5
Operating System (With Version):Windows 10 Home
Laptop Type:gaming
Display
Resolution:1920 x 1080
Display Size (In Inches):15.6
Connectivity
Connectivity:v4.2
Memory
Ram Included (In Gb):8 GB
Ram Type:DDR4
Physical Specifications
Laptop Weight (In Kgs):2.3
Laptop Dimension (In Mm):363.4 x 255 x 25.9
Processor
Processor Model Name:AMD Ryzen 5 Quad Core -
3550H
Clock Speed:2.1 GHz
Cores:4
Graphics Processor:NVIDIA GeForce GTX 1650
Cache L3:6 MB
Boost Clock Speed:3.7 GHz
Storage
Storage Drive Type:HDD
Storage Drive Capacity:1 TB
Power
Battery Backup (In Hours):10 hours
Battery Type:4-cell Li-ion
Power Supply:135 W
Sound
Speakers:Yes


Getting data for product 2

Basic Information
Model Name:Aspire 7 Ryzen 5
Operating System (With Version):Windows 10 Home
Laptop Type:gaming
Display
Resolution:1920 x 1080
Display Size (In Inches):15.6
Connectivity
Connectivity:v5.0
Memory
Ram Included (In Gb):8 GB
Ram Type:DDR4
Physical Specif

8.Write a python program to scrape the details for all billionaires from www.forbes.com.
Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”,
“Industry”

In [19]:
def report(rank, name, net_worth, age, citizen, source, industry):
    df = pd.read_excel("Q8.xlsx")
    #Saving data into excel sheet
    data = {"Rank":[rank],"Name":[name],"Net Worth":[net_worth],
            "Age":[age],"Citizenship":[citizen],"Source":[source],
            "Industry":[industry]
            }
    df = df.append(pd.DataFrame(data))
    writer = pd.ExcelWriter("Q8.xlsx")
    df.to_excel(writer,index=False)
    writer.save()

if __name__ == "__main__":
    browser = webdriver.Chrome()
    wait = WebDriverWait(browser, 600)

    browser.get("https://www.forbes.com")
    browser.execute_script("window.close();")

    #Goto Billionaires list
    browser.get("https://www.forbes.com/billionaires/")
    time.sleep(3)

    ranks = browser.find_elements_by_class_name("rank")
    names = browser.find_elements_by_class_name("personName")
    nets = browser.find_elements_by_class_name("netWorth")
    ages = browser.find_elements_by_class_name("age")
    citizens = browser.find_elements_by_class_name("countryOfCitizenship")
    sources = browser.find_elements_by_class_name("source")
    categories = browser.find_elements_by_class_name("category")

    for rank,name,net,age,citizen,source,category in zip(ranks,names,nets,ages,citizens,sources,categories):
        print(rank.text, name.text, net.text, age.text, citizen.text, source.text, category.text)
        print()
        print()
        report(rank.text, name.text, net.text, age.text, citizen.text, source.text, category.text)

RANK NAME NET WORTH AGE COUNTRY / TERRITORY SOURCE INDUSTRY


1. Jeff Bezos $177 B 57 United States Amazon Technology


2. Elon Musk $151 B 49 United States Tesla, SpaceX Automotive


3. Bernard Arnault & family $150 B 72 France LVMH Fashion & Retail


4. Bill Gates $124 B 65 United States Microsoft Technology


5. Mark Zuckerberg $97 B 36 United States Facebook Technology


6. Warren Buffett $96 B 90 United States Berkshire Hathaway Finance & Investments


7. Larry Ellison $93 B 76 United States software Technology


8. Larry Page $91.5 B 48 United States Google Technology


9. Sergey Brin $89 B 47 United States Google Technology


10. Mukesh Ambani $84.5 B 63 India diversified Diversified


11. Amancio Ortega $77 B 85 Spain Zara Fashion & Retail


12. Francoise Bettencourt Meyers & family $73.6 B 67 France L'Oréal Fashion & Retail


13. Zhong Shanshan $68.9 B 66 China beverages, pharmaceuticals Food & Beverage


14. Steve Ballmer $68.7 B 65 United States Microsoft Technology


15. Ma

9.Write a program to extract at least 500 Comments, Comment upvote and time when comment
was posted from any YouTube Video.

In [24]:
def report(comment,upvote,time):
    df = pd.read_excel("Q9.xlsx")

    #Saving data into excel sheet
    data = {"Comment":[comment],"Upvotes":[upvote],"Time":[time]}
    df = df.append(pd.DataFrame(data))
    writer = pd.ExcelWriter("Q9.xlsx")
    df.to_excel(writer,index=False)
    writer.save()

if __name__=="__main__":
    browser = webdriver.Chrome()
    browser.get("https://www.youtube.com/watch?v=Pkh8UtuejGw")
    time.sleep(5)

    count=1
    while(count<=50):
        if count==1:
            browser.find_element_by_tag_name("body").send_keys(Keys.PAGE_DOWN)
            time.sleep(3)
        time.sleep(2)
        browser.find_element_by_tag_name("body").send_keys(Keys.END)
        count+=1

    browser.find_element_by_tag_name("body").send_keys(Keys.PAGE_UP)
    html = browser.page_source
    soup = BeautifulSoup(html,"html.parser")
    tags = soup.find_all("ytd-comment-renderer",{"id":"comment"},limit=500)
    upvotes = soup.find_all("span",{"id":"vote-count-middle"},limit=500)
    times = soup.find_all("yt-formatted-string",{"class":"published-time-text above-comment style-scope ytd-comment-renderer"},limit=500)
    print(len(tags), len(upvotes), len(times))
    for tag, upvote, t in zip(tags, upvotes, times):
        report(re.sub("\s+"," ",tag.text),
                re.sub("\s+"," ",upvote.text), t.text)

500 500 500


10.Write a python program to scrape a data for all available Hostels from
https://www.hostelworld.com/ in “London” location. You have to scrape hostel name,
distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms
from price, facilities and property description. 

In [25]:
def getName(div):
    try:
        return div.find("a").get_text()
    except:
        return "-"

def getDistance(div):
    try:
        return div.find("span",{"class":"description"}).get_text()
    except:
        return "-"

def getRatings(div):
    try:
        return div.find("div",{"class":"score orange big"}).get_text()
    except:
        return "-"

def getTotalReviews(div):
    try:
        return div.find("div",{"class":"reviews"}).get_text()
    except:
        return "-"

def getOverallReviews(div):
    try:
        return div.find("div",{"class":"keyword"}).get_text()
    except:
        return "-"

def privatePrice(div):
    try:
        return div.find("div",{"class":"price title-5"}).get_text()
    except:
        return "-"

def dormsPrice(div):
    try:
        return div.find("div",{"class":"price title-5"}).findNext("div").get_text()
    except:
        return "-"

def getFacilities(div):
    try:
        return div.find("div",{"class":"facilities-label facilities"}).get_text()
    except:
        return "-"

def getDescription(div):
    try:
        return div.find("div",{"class":"rating-factors prop-card-tablet rating-factors small"}).get_text()
    except:
        return "-"

if __name__=="__main__":
    browser = webdriver.Chrome()
    browser.get("https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3&from=2021-06-27&to=2021-06-30&guests=2&page=1")
    wait = WebDriverWait(browser, 600)  

    time.sleep(10)

    html = browser.page_source
    soup = BeautifulSoup(html,"html.parser")

    divs = soup.find_all("div",{"class":"property-card"})
    divs.pop(0)
    divs.pop(0)

    for div in divs:
        name = getName(div)
        distance = getDistance(div)
        ratings = getRatings(div)
        total_reviews = getTotalReviews(div)
        overall_reviews = getOverallReviews(div)
        privates_price = privatePrice(div)
        dorms_price = dormsPrice(div)
        facilities = getFacilities(div)
        description = getDescription(div)

        name = re.sub("\s+"," ",name)
        distance = re.sub("\s+"," ",distance)
        ratings = re.sub("\s+"," ",ratings)
        total_reviews = re.sub("\s+"," ",total_reviews)
        overall_reviews = re.sub("\s+"," ",overall_reviews)
        privates_price = re.sub("\s+"," ",privates_price)
        dorms_price = re.sub("\s+"," ",dorms_price)
        facilities = re.sub("\s+"," ",facilities)
        description = re.sub("\s+"," ",description)


        print(name)
        print(distance)
        print(ratings)
        print(total_reviews)
        print(overall_reviews)
        print(privates_price)
        print(dorms_price)
        print(facilities)
        print(description)
        print()
        print()

 1/ 30 
Hostel - 3km from city centre
 7.5 
6729 Total Reviews
Very Good
 Rs8156 
Dorms From Rs1962 
 Free WiFi Follows Covid-19 sanitation guidance
 Perfect Location Marvellous Staff Awesome Cleanliness


 1/ 30 
Hostel - 4.7km from city centre
 7.9 
204 Total Reviews
Very Good
 Rs4956 
Dorms From Rs1239 
 Free WiFi Follows Covid-19 sanitation guidance
 Wonderful Location Superb Staff Excellent Cleanliness


 1/ 14 
Hostel - 3.6km from city centre
-
13 Total Reviews
No Rating
 Rs5575 
 No Dorms Available
 Free WiFi 
 


 1/ 22 
Hostel - 0.5km from city centre
 8.7 
183 Total Reviews
Fabulous
 Rs14006 
Dorms From Rs1617 
 Free WiFi Follows Covid-19 sanitation guidance
 Perfect Location Superb Staff Excellent Cleanliness


 1/ 29 
Hostel - 3.6km from city centre
 9.3 
13123 Total Reviews
Superb
 Rs8852 
Dorms From Rs2687 
 Free WiFi Follows Covid-19 sanitation guidance
 Perfect Location Superb Staff Fantastic Cleanliness


 1/ 30 
Hostel - 1.7km from city centre
 8.0 
3930 Total Reviews